-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Exercise #3 - Create Fact & Dim Tables

Now that the three years of orders are combined into a single dataset, we can begin the processes of transforming the data.

In the one record, there are actually four sub-datasets:
* The order itself which is the aggregator of the other three datasets.
* The line items of each order which includes the price and quantity of each specific item.
* The sales rep placing the order.
* The customer placing the order - for the sake of simplicity, we will **not** break this dataset out and leave it as part of the order.

What we want to do next, is to extract all that data into their respective datasets (except the customer data). 

In other words, we want to normalize the data, in this case, to reduce data duplication.

This exercise is broken up into 5 steps:
* Exercise 3.A - Create & Use Database
* Exercise 3.B - Load & Cache Batch Orders
* Exercise 3.C - Extract Sales Reps
* Exercise 3.D - Extract Orders
* Exercise 3.E - Extract Line Items

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Setup Exercise #3</h2>

To get started, we first need to configure your Registration ID and then run the setup notebook.

### Setup - Registration ID

In the next command, please update the variable **`registration_id`** with the Registration ID you received when you signed up for this project.

For more information, see [Registration ID]($./Registration ID)

In [0]:
registration_id = "3339094"

### Setup - Run the exercise setup

Run the following cell to setup this exercise, declaring exercise-specific variables and functions.

In [0]:
%run ./_includes/Setup-Exercise-03

Variable/Function,Description
username,dakota.murdock@wavicledata.com
,This is the email address that you signed into Databricks with
working_dir,dbfs:/user/dakota.murdock@wavicledata.com/dbacademy/developer-foundations-capstone
,This is the directory in which all work should be conducted
user_db,dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone
,The name of the database you will use for this project.
batch_source_path,dbfs:/user/dakota.murdock@wavicledata.com/dbacademy/developer-foundations-capstone/batch_orders_dirty.delta
,"The location of the combined, raw, batch of orders."
orders_table,orders
,The name of the orders table.


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.A - Create &amp; Use Database</h2>

By using a specific database, we can avoid contention to commonly named tables that may be in use by other users of the workspace.

**In this step you will need to:**
* Create the database identified by the variable **`user_db`**
* Use the database identified by the variable **`user_db`** so that any tables created in this notebook are **NOT** added to the **`default`** database

**Special Notes**
* Do not hard-code the database name - in some scenarios this will result in validation errors.
* For assistence with the SQL command to create a database, see <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-database.html" target="_blank">CREATE DATABASE</a> on the Databricks docs website.
* For assistence with the SQL command to use a database, see <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-usedb.html" target="_blank">USE DATABASE</a> on the Databricks docs website.

### Implement Exercise #3.A

Implement your solution in the following cell:

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {user_db};")
spark.sql(f"USE {user_db};")

Out[15]: DataFrame[]

### Reality Check #3.A
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_a()

Points,Test,Result
1,Using DBR 9.1 & Proper Cluster Configuration,
1,Valid Registration ID,
1,The current database is dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone,


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.B - Load &amp; Cache Batch Orders</h2>

Next, we need to load the batch orders from the previous exercise and then cache them in preparation to transform the data later in this exercise.

**In this step you will need to:**
* Load the delta dataset we created in the previous exercise, identified by the variable **`batch_source_path`**.
* Using that same dataset, create a temporary view identified by the variable **`batch_temp_view`**.
* Cache the temporary view.

### Implement Exercise #3.B

Implement your solution in the following cell:

In [0]:
batch_orders = spark.read.format("delta").load(batch_source_path)
batch_orders.createOrReplaceTempView(batch_temp_view)
spark.sql(f"CACHE TABLE {batch_temp_view};")

Out[17]: DataFrame[]

### Reality Check #3.B
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_b()

Points,Test,Result
1,The current database is dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone,
1,The table batched_orders exists,
1,The table batched_orders is a temp view,
1,The table batched_orders is cached,
1,"Expected 1,175,870 records",


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.C - Extract Sales Reps</h2>

Our batched orders from Exercise #2 contains thousands of orders and with every order, is the name, SSN, address and other information on the sales rep making the order.

We can use this data to create a table of just our sales reps.

If you consider that we have only ~100 sales reps, but thousands of orders, we are going to have a lot of duplicate data in this space.

Also unique to this set of data, is the fact that social security numbers were not always sanitized meaning sometime they were formatted with hyphens and in other cases they were not - this is something we will have to address here.

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* The SSN numbers have errors in them that we want to track - add the **`boolean`** column **`_error_ssn_format`** - for any case where **`sales_rep_ssn`** has a hypen in it, set this value to **`true`** otherwise **`false`**
* Convert various columns from their string representation to the specified type:
  * The column **`sales_rep_ssn`** should be represented as a **`Long`** (Note: You will have to first clean the column by removing extreneous hyphens in some records)
  * The column **`sales_rep_zip`** should be represented as an **`Integer`**
* Remove the columns not directly related to the sales-rep record:
  * Unrelated ID columns: **`submitted_at`**, **`order_id`**, **`customer_id`**
  * Shipping address columns: **`shipping_address_attention`**, **`shipping_address_address`**, **`shipping_address_city`**, **`shipping_address_state`**, **`shipping_address_zip`**
  * Product columns: **`product_id`**, **`product_quantity`**, **`product_sold_price`**
* Because there is one record per product ordered (many products per order), not to mention one sales rep placing many orders (many orders per sales rep), there will be duplicate records for our sales reps. Remove all duplicate records, making sure to exclude **`ingest_file_name`** and **`ingested_at`** from the evaluation of duplicate records
* Load the dataset to the managed delta table **`sales_rep_scd`** (identified by the variable **`sales_reps_table`**)

**Additional Requirements:**<br/>
The schema for the **`sales_rep_scd`** table must be:
* **`sales_rep_id`**:**`string`**
* **`sales_rep_ssn`**:**`long`**
* **`sales_rep_first_name`**:**`string`**
* **`sales_rep_last_name`**:**`string`**
* **`sales_rep_address`**:**`string`**
* **`sales_rep_city`**:**`string`**
* **`sales_rep_state`**:**`string`**
* **`sales_rep_zip`**:**`integer`**
* **`ingest_file_name`**:**`string`**
* **`ingested_at`**:**`timestamp`**
* **`_error_ssn_format`**:**`boolean`**

### Implement Exercise #3.C

Implement your solution in the following cell:

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone.db/sales_reps", True)

from pyspark.sql.functions import *

# # Load from temp view
df_batched_orders = spark.sql(f"SELECT * FROM {batch_temp_view}")

# # Perform transformations
df_sales_reps = (df_batched_orders.withColumn("_error_ssn_format", col("sales_rep_ssn").contains('-')) # Create column identifying if there was an error with the ssn format
                                      .withColumn("sales_rep_ssn", regexp_replace("sales_rep_ssn", "-", "").cast("long")) # Remove hyphens and cast ssn column to long
                                      .withColumn("sales_rep_zip", col("sales_rep_zip").cast("integer")) # Cast zip column to integer
                                      .drop("submitted_at", "order_id", "customer_id", "shipping_address_attention", "shipping_address_address", "shipping_address_city", "shipping_address_state", "shipping_address_zip", 
                                           "product_id", "product_quantity", "product_sold_price") # Drop unrelated columns
                                      .dropDuplicates(['sales_rep_id']) # Remove duplicate sales reps
                    )

# # Load dataset to delta
df_sales_reps.write.format("delta").saveAsTable(sales_reps_table)

### Reality Check #3.C
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_c()

PYTHON ERROR Invalid argument, not a string or column: 1636991566781.1484 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991567034.866 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991567249.093 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991567329.6929 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991567525.2673 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991568087.7048 of type <class 'float'>. For column literals, use 'li

Points,Test,Result
1,The current database is dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone,
1,The table sales_reps exists,
1,The table sales_reps is a managed table,
1,Using the Delta file format,
1,Schema is valid,
1,Expected 93 records,
1,Expected _error_ssn_format record count to be 17,


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.D - Extract Orders</h2>

Our batched orders from Exercise 02 contains one line per product meaning there are multiple records per order.

The goal of this step is to extract just the order details (excluding the sales rep and line items)

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* Convert various columns from their string representation to the specified type:
  * The column **`submitted_at`** is a "unix epoch" (number of seconds since 1970-01-01 00:00:00 UTC) and should be represented as a **`Timestamp`**
  * The column **`shipping_address_zip`** should be represented as an **`Integer`**
* Remove the columns not directly related to the order record:
  * Sales reps columns: **`sales_rep_ssn`**, **`sales_rep_first_name`**, **`sales_rep_last_name`**, **`sales_rep_address`**, **`sales_rep_city`**, **`sales_rep_state`**, **`sales_rep_zip`**
  * Product columns: **`product_id`**, **`product_quantity`**, **`product_sold_price`**
* Because there is one record per product ordered (many products per order), there will be duplicate records for each order. Remove all duplicate records, making sure to exclude **`ingest_file_name`** and **`ingested_at`** from the evaluation of duplicate records
* Add the column **`submitted_yyyy_mm`** which is a **`string`** derived from **`submitted_at`** and is formatted as "**yyyy-MM**".
* Load the dataset to the managed delta table **`orders`** (identified by the variable **`orders_table`**)
  * In thise case, the data must also be partitioned by **`submitted_yyyy_mm`**

**Additional Requirements:**
* The schema for the **`orders`** table must be:
  * **`submitted_at:timestamp`**
  * **`submitted_yyyy_mm`** using the format "**yyyy-MM**"
  * **`order_id:string`**
  * **`customer_id:string`**
  * **`sales_rep_id:string`**
  * **`shipping_address_attention:string`**
  * **`shipping_address_address:string`**
  * **`shipping_address_city:string`**
  * **`shipping_address_state:string`**
  * **`shipping_address_zip:integer`**
  * **`ingest_file_name:string`**
  * **`ingested_at:timestamp`**

### Implement Exercise #3.D

Implement your solution in the following cell:

In [0]:
from pyspark.sql.functions import *

dbutils.fs.rm("dbfs:/user/hive/warehouse/dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone.db/orders", True)

# Load from temp view
df_batched_orders = spark.sql(f"SELECT * FROM {batch_temp_view}")

# Perform transformations
df_orders = (df_batched_orders.withColumn("submitted_at", from_unixtime("submitted_at").cast("timestamp")) # Convert from unix time and cast column to timestamp
                                      .withColumn("submitted_yyyy_mm", date_format("submitted_at", "yyyy-MM")) # Created new year/month column from submitted_at column
                                      .withColumn("shipping_address_zip", col("shipping_address_zip").cast("integer")) # Cast zip column to integer
                                      .drop("sales_rep_ssn", "sales_rep_first_name", "sales_rep_last_name", "sales_rep_address", "sales_rep_city", "sales_rep_state", "sales_rep_zip",
                                           "product_id", "product_quantity", "product_sold_price") # Drop unrelated columns
                                      .dropDuplicates(['order_id']) # Drop duplicate order numbers
                    )

# Load dataset to delta
df_orders.write.format("delta").saveAsTable(orders_table, partitionBy="submitted_yyyy_mm")

### Reality Check #3.D
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_d()

PYTHON ERROR Invalid argument, not a string or column: 1636991610887.321 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991611242.6484 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991611869.9744 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991612018.2441 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991612155.8555 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991613153.0186 of type <class 'float'>. For column literals, use 'l

Points,Test,Result
1,The current database is dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone,
1,The table orders exists,
1,The table orders is a managed table,
1,Using the Delta file format,
1,Schema is valid,
1,"Expected 195,698 records",
1,Non-null (properly parsed) submitted_at,
1,Partitioned by submitted_yyyy_mm,
1,Found 36 partitions,


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.E - Extract Line Items</h2>

Now that we have extracted sales reps and orders, we next want to extract the specific line items of each order.

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* Retain the following columns (see schema below)
  * The correlating ID columns: **`order_id`** and **`product_id`**
  * The two product-specific columns: **`product_quantity`** and **`product_sold_price`**
  * The two ingest columns: **`ingest_file_name`** and **`ingested_at`**
* Convert various columns from their string representation to the specified type:
  * The column **`product_quantity`** should be represented as an **`Integer`**
  * The column **`product_sold_price`** should be represented as an **`Decimal`** with two decimal places as in **`decimal(10,2)`**
* Load the dataset to the managed delta table **`line_items`** (identified by the variable **`line_items_table`**)

**Additional Requirements:**
* The schema for the **`line_items`** table must be:
  * **`order_id`**:**`string`**
  * **`product_id`**:**`string`**
  * **`product_quantity`**:**`integer`**
  * **`product_sold_price`**:**`decimal(10,2)`**
  * **`ingest_file_name`**:**`string`**
  * **`ingested_at`**:**`timestamp`**

### Implement Exercise #3.E

Implement your solution in the following cell:

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import DecimalType

dbutils.fs.rm("dbfs:/user/hive/warehouse/dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone.db/line_items", True)

# Load from temp view
df_batched_orders = spark.sql(f"SELECT * FROM {batch_temp_view}")

# Perform transformations
df_line_items = (df_batched_orders.select("order_id", "product_id", "product_quantity", "product_sold_price", "ingest_file_name", "ingested_at") # Select relevant columns
                                  .withColumn("product_quantity", col("product_quantity").cast("integer")) # Cast product_quantity column to integer type
                                  .withColumn("product_sold_price", col("product_sold_price").cast(DecimalType(10,2))) # Cast product_sold_price column to decimal type with precision 10 and 2 decimal places
                )

# Load dataset to delta
df_line_items.write.format("delta").saveAsTable(line_items_table)


### Reality Check #3.E
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_e()

PYTHON ERROR Invalid argument, not a string or column: 1636991658718.8657 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991659246.337 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991659691.0608 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991659790.256 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991659910.607 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991660452.4314 of type <class 'float'>. For column literals, use 'lit

Points,Test,Result
1,The current database is dbacademy_dakota_murdock_wavicledata_com_developer_foundations_capstone,
1,The table line_items exists,
1,The table line_items is a managed table,
1,Using the Delta file format,
1,Schema is valid,
1,"Expected 1,175,870 records",


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3 - Final Check</h2>

Run the following command to make sure this exercise is complete:

In [0]:
reality_check_03_final()

PYTHON ERROR Invalid argument, not a string or column: 1636991660556.9578 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991660578.8606 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991660603.02 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991660623.0173 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991660643.66 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.
PYTHON ERROR Invalid argument, not a string or column: 1636991660663.9404 of type <class 'float'>. For column literals, use 'lit'

Points,Test,Result
1,Reality Check 03.A passed,
1,Reality Check 03.B passed,
1,Reality Check 03.C passed,
1,Reality Check 03.D passed,
1,Reality Check 03.E passed,


-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>